In [1]:
import sys
import os

sys.path.append(os.path.abspath(".."))

In [2]:
from torch.utils.data import DataLoader
from src import SCUTDataset

from src import load_model
from dataclasses import dataclass

d:\marchine_learning\Projet\Deep_learning Project\OCR project\SCUT-CTW\trocr_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
@dataclass(frozen=True)
class TrainingConfig:
    BATCH_SIZE:    int = 16
    EPOCHS:        int = 25
    LEARNING_RATE: float = 0.000005

@dataclass(frozen=True)
class DatasetConfig:
    DATA_ROOT:     str = 'data/scut_data'

@dataclass(frozen=True)
class ModelConfig:
    """
    Check this for complete pretrained Transformer Configuration.
    https://huggingface.co/docs/transformers/v4.29.1/en/main_classes/configuration#transformers.PretrainedConfig
    """
    MODEL_NAME: str = 'microsoft/trocr-base-handwritten'

In [4]:
processor, model, device = load_model(model_name = ModelConfig.MODEL_NAME)

Loading weights: 100%|██████████| 478/478 [00:03<00:00, 121.92it/s, Materializing param=encoder.layernorm.weight]                                      
VisionEncoderDecoderModel LOAD REPORT from: microsoft/trocr-base-handwritten
Key                         | Status  | 
----------------------------+---------+-
encoder.pooler.dense.bias   | MISSING | 
encoder.pooler.dense.weight | MISSING | 

Notes:
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


In [5]:
train_dataset = SCUTDataset(
    root_dir="D:/marchine_learning/Projet/Deep_learning Project/OCR project/SCUT-CTW/data/scut_data/scut_train/",     # adapte ton chemin
    ann_path="D:/marchine_learning/Projet/Deep_learning Project/OCR project/SCUT-CTW/data/scut_data/scut_train.txt",
    processor= processor,
)
train_dataloader = DataLoader(
    train_dataset, batch_size=TrainingConfig.BATCH_SIZE, shuffle=True
)

In [6]:
# Test one bacth
batch = next(iter(train_dataloader))

print(batch["pixel_values"].shape)
print(batch["labels"].shape)

torch.Size([16, 3, 384, 384])
torch.Size([16, 128])


In [7]:
#Test mini forward
pixel_values = batch["pixel_values"].to(device) 
labels = batch["labels"].to(device) 
outputs = model(pixel_values=pixel_values, labels=labels) 
outputs.loss

tensor(9.9390, grad_fn=<NllLossBackward0>)